   # Sandbox: Ignore this
   This notebook contains little experiments on doing things with python, jupyter, ipython et cetera
   It's not a central part of the project.
    
   Mostly ad hoc code snippets.  And not documented well

In [1]:
import os
import sys
import IPython
import pandas as pd

In [2]:
# snippet to see the source code for a fuction
import inspect
import nbformat
fname = "nbformat.convert"
lines = inspect.getsource(eval(fname))
print(lines)

def convert(nb, to_version):
    """Convert a notebook node object to a specific version.  Assumes that
    all the versions starting from 1 to the latest major X are implemented.
    In other words, there should never be a case where v1 v2 v3 v5 exist without
    a v4.  Also assumes that all conversions can be made in one step increments
    between major versions and ignores minor revisions.

    Parameters
    ----------
    nb : NotebookNode
    to_version : int
        Major revision to convert the notebook to.  Can either be an upgrade or
        a downgrade.
    """

    # Get input notebook version.
    (version, version_minor) = get_version(nb)

    # Check if destination is target version, if so return contents
    if version == to_version:
        return nb

    # If the version exist, try to convert to it one step at a time.
    elif to_version in versions:

        # Get the the version that this recursion will convert to as a step 
        # closer to the final revision. 

In [4]:
# to get the names of notebook files in a directory
def get_project_notebooks():
    from nbformat import read, NO_CONVERT
    import os
    import datetime
    proj_nbs = []
    nb_fnames = [f for f in os.listdir() if f.endswith(".ipynb")]
    for fn in nb_fnames:
        with open(fn) as fp:
            notebook = read(fp, as_version=NO_CONVERT)
        cells = notebook['cells']
        if not cells:
            continue
        src =cells[0]["source"]
        if not src:
            continue
        lines = src.split("\n")
        if lines[0].startswith("# "):
            proj_nbs.append(fn)
    return(proj_nbs)
get_project_notebooks()

['combiningData.ipynb',
 'DataSources.ipynb',
 'fastpages-howto.ipynb',
 'fred.ipynb',
 'google.ipynb',
 'Intro_Overview.ipynb',
 'umich_cs.ipynb',
 'weekly_claims.ipynb']

In [5]:
# print the source code for a cell
def print_source(cell):
    print("type cell= {0}".format(type(cell)))
    src = cell["source"]
    lines = src.split("\n")
    print("{0} lines".format(len(lines)))
    for i,line in enumerate(lines):
        print("({0}){1}".format(i, line))

# run notebook code
def run_nb_code(nbfile):
    from nbformat import read, NO_CONVERT
    import os
    import datetime
    res = "Start time: " + str(datetime.datetime.now())
    with open(nbfile) as fp:
        notebook = read(fp, NO_CONVERT)
    cells = notebook['cells']
    code_cells = [c for c in cells if c['cell_type'] == 'code']
    for i, cell in enumerate(code_cells):
        src = cell['source']
        #print("{0}<{1}>".format(li, line))                
        try:
            exec(src, globals(), locals())
        except Exception as e:
            print("Error on cell {0}\n".format(i))
            print(src)
            print_source(cell)
            exc_type, exc_obj, exc_tb = sys.exc_info()
            import traceback
            traceback.print_tb(exc_tb)
            fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
            print(exc_type, fname, exc_tb.tb_lineno)
            return((exc_type, exc_obj, exc_tb, fname))
    res += "end: " + str(datetime.datetime.now())
    return res
%run bar.ipynb

In [ ]:
%run bar.ipynb

In [6]:
nbs = get_project_notebooks()
nbs = [ 'fred.ipynb',
         'DataSources.ipynb',
    'google.ipynb',
   'Intro_Overview.ipynb',
     'weekly_claims.ipynb', 
    'umich_cs.ipynb',
      'combiningData.ipynb'
       ]
for nb in nbs:
    print(nb)
    %run $nb

fred.ipynb
version 3.7.9 of Python
version 5.8.0 of IPython
version 1.1.3 of pd
version 5.0.8 of nbformat
version 1.19.2 of np
version 3.3.3 of mpl
0.4.2
USPRIV, rows= 166
NPPTTL, rows= 166
ICSA, rows= 724
CCSA, rows= 723
JTS1000JOL, rows= 165
JTS1000HIL, rows= 165
JTS1000TSL, rows= 165
CSCICP03USM665S, rows= 166
BSCICP03USM665S, rows= 166
UNRATE, rows= 166
UNEMPLOY, rows= 166
CLF16OV, rows= 166
UEMP27OV, rows= 166
U6Rate, rows= 166
CIVPART, rows= 166
LNS12032194, rows= 166
     realtime_start        date      value series_id         description  \
1053     2020-11-05  2020-10-31     751000      ICSA      Initial Claims   
1054     2020-11-12  2020-11-07     709000      ICSA      Initial Claims   
1778     2020-11-19  2020-11-07  6.372e+06      CCSA   Continued Claims    
1055     2020-11-19  2020-11-14     742000      ICSA      Initial Claims   

          dtime  
1053 2020-10-31  
1054 2020-11-07  
1778 2020-11-07  
1055 2020-11-14  
           date  USPRIV    ICSA JTS1000JOL
0    20

<Figure size 2000x1000 with 16 Axes>

<Figure size 2000x1000 with 16 Axes>

<Figure size 1200x600 with 1 Axes>

<Figure size 2000x1000 with 1 Axes>

2020-11-24 01:52:57.115920
Intro_Overview.ipynb
weekly_claims.ipynb
version 3.7.9 of Python
version 5.8.0 of IPython
version 1.1.3 of pd
version 5.0.8 of nbformat
version 1.19.2 of np
version 3.3.3 of mpl
version 0.4.2 of fredapi
version 0.7.1 of pn
version 1.2.0 of xlrd
version 3.141.0 of selenium
version 3.14.1 of webdriver
new file: ./data/temp\weekly_pandemic_claims.xlsx
          date    PUA IC      PUA CC    PEUC CC
27  2020-10-10  337228.0  10152753.0  3308937.0
28  2020-10-17  345440.0  10324779.0  3691720.0
29  2020-10-24  359044.0   9332610.0  3983613.0
30  2020-10-31  361959.0   9433127.0  4143389.0
31  2020-11-07  298154.0         NaN        NaN
umich_cs.ipynb
version 3.7.9 of Python
version 5.8.0 of IPython
version 1.1.3 of pd
version 5.0.8 of nbformat
version 1.19.2 of np
version 3.3.3 of mpl
version 0.4.2 of fredapi
version 0.7.1 of pn
version 1.2.0 of xlrd
version 3.141.0 of selenium
version 3.14.1 of webdriver
version 2.2.1 of re
set()
{'sca-table5-on-2020-Nov-24.csv'}

<Figure size 1200x600 with 2 Axes>

<Figure size 1600x400 with 1 Axes>

<Figure size 1400x800 with 2 Axes>

In [ ]:
# traceback opbeimport traceback
(exc_type, exc_obj, exc_tb, fname) = res
traceback.print_tb(exc_tb)

In [ ]:
all_nb_filenames = [f for f in os.listdir() if f.endswith(".ipynb")]
print(all_nb_filenames)
proj_nb_filenames = ['fred.ipynb', 'combiningData.ipynb', 'DataSources.ipynb', 'fastpages-howto.ipynb', 
                     'foo.ipynb', 'fred.ipynb', 'google.ipynb', 'Intro_Overview.ipynb', 'umich_cs.ipynb', 'weekly_claims.ipynb']

In [ ]:
import IPython
app = IPython.Application.instance()
dir(app.kernel)
app.kernel.do_shutdown(restart=True)
#app.kernel.do_shutdown(True) 
print(datetime.datetime.now())

In [ ]:
import re
head, tail = os.path.split(downloaded_fpath)
base, ext = os.path.splitext(tail)
pattern = "(.*)(\(.*\))"
match = re.match(pattern, base)
if not match:
    msg = "Error cleaning {0}\nNo match for pattern: <{1}>".format(fpath, pattern)
    raise Exception(msg)
new_base = match.group(1).strip()
new_fname = new_base + ext
return new_fname

In [ ]:
def print_vars(globs=True, locs=True, 
               filters={"exclude":("^_",)}):
    import re
    dicts = {"globs":globals().keys(),
            "locs":locals().keys()}
    for k,v in dicts.items():
        print("k={0}, eval(k)= {1}".format(k, eval(k)))
        if eval(k):
            y = [x for x in dicts[k]]
            if filters:
                if filters["exclude"]:
                    for filt in filters["exclude"]:
                        print("excluding: {0}".format(filt))
                        y = [x for x in y if not re.search(filt, x)]
            print("{0}= {1}".format(k,y))
        del y
print_vars()

In [ ]:
z